In [1]:
!pip install pydot

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install graphviz

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [6]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

# Sequential API

In [7]:
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [8]:
seq_model = keras.Sequential([
    keras.layers.Dense(10, input_shape=[8,], activation='relu'),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(1)])

seq_model.compile(loss='mean_squared_error', optimizer='sgd')
history = seq_model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

mse_test = seq_model.evaluate(X_test, y_test)
X_new = X_test[:3] # pretend these are new instances
y_pred = seq_model.predict(X_new)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 11610 samples, validate on 3870 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
11610/11610 [==============================] - 3s 257us/sample - loss: 0.9027 - val_loss: 4.9606
Epoch 2/10
11610/11610 [==============================] - 0s 34us/sample - loss: 0.5192 - val_loss: 12.5950
Epoch 3/10
11610/11610 [==============================] - 0s 34us/sample - loss: 0.5083 - val_loss: 0.3982
Epoch 4/10
11610/11610 [==============================] - 0s 34us/sample - loss: 0.4204 - val_loss: 0.3848
Epoch 5/10
11610/11610 [==============================] - 0s 34us/sample - loss: 0.4086 - val_loss: 0.3754
Epoch 6/10
11610/11610 [==============================] - 0s 34us/sample - loss: 0.4010 - val_loss: 0.3691
Epoch 7/10
11610/11610 [==============================] - 0s 34us/sample - loss: 0.3933 - val_loss: 0.3624
Epoch 8/10
11610/11610 [========

In [9]:
keras.utils.plot_model(seq_model, "my_seq_model.png", show_shapes=True)

In [7]:
seq_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                90        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________


# Hyperparamter search

In [10]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [11]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [14]:
np.arange(1, 100)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [15]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 7740 samples, validate on 3870 samples
Epoch 1/100
7740/7740 [==============================] - 1s 76us/sample - loss: 0.6430 - val_loss: 0.4172
Epoch 2/100
7740/7740 [==============================] - 0s 45us/sample - loss: 0.3879 - val_loss: 2.0975
Epoch 3/100
7740/7740 [==============================] - 0s 45us/sample - loss: 0.4177 - val_loss: 0.5990
Epoch 4/100
7740/7740 [==============================] - 0s 45us/sample - loss: 0.3740 - val_loss: 8.7048
Epoch 5/100
7740/7740 [==============================] - 0s 45us/sample - loss: 0.3989 - val_loss: 1.2099
Epoch 6/100
7740/7740 [==============================] - 0s 45us/sample - loss: 0.3569 - val_loss: 0.3410
Epoch 7/100
7740/7740 [==============================] - 0s 45us/sample - loss: 0.3368 - val_loss: 0.6392
Epoch 8/100
7740/7740 [==============================] - 0s 45us/sample - loss: 0.3338 - val_loss: 0.5411
Epoch 9/100
7740/7740 [==============================] - 0s 44us/sample - loss: 0.3278 - val_loss: 0.629

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fa3bc2aeda0>,
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_hidden': [0, 1, 2, 3], 'n_neurons': array([ 1,  2, ..., 98, 99]), 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa3bc2aaa90>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [16]:
rnd_search_cv.best_params_

{'learning_rate': 0.01858105582198142, 'n_hidden': 3, 'n_neurons': 91}

In [18]:
rnd_search_cv.best_score_

-0.29848999126937037

In [19]:
rnd_search_cv_valid_on_train = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv_valid_on_train.fit(X_train, y_train, epochs=100,
                  validation_data=(X_train, y_train),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 7740 samples, validate on 11610 samples
Epoch 1/100
7740/7740 [==============================] - 2s 286us/sample - loss: 2.6533 - val_loss: 1.0742
Epoch 2/100
7740/7740 [==============================] - 1s 74us/sample - loss: 0.8570 - val_loss: 0.7339
Epoch 3/100
7740/7740 [==============================] - 1s 74us/sample - loss: 0.6973 - val_loss: 0.6615
Epoch 4/100
7740/7740 [==============================] - 1s 74us/sample - loss: 0.6388 - val_loss: 0.6227
Epoch 5/100
7740/7740 [==============================] - 1s 73us/sample - loss: 0.6085 - val_loss: 0.5997
Epoch 6/100
7740/7740 [==============================] - 1s 73us/sample - loss: 0.5804 - val_loss: 0.5687
Epoch 7/100
7740/7740 [==============================] - 1s 73us/sample - loss: 0.5549 - val_loss: 0.5463
Epoch 8/100
7740/7740 [==============================] - 1s 74us/sample - loss: 0.5339 - val_loss: 0.5247
Epoch 9/100
7740/7740 [==============================] - 1s 73us/sample - loss: 0.5130 - val_loss: 0.5

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fa3bc2aeda0>,
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_hidden': [0, 1, 2, 3], 'n_neurons': array([ 1,  2, ..., 98, 99]), 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fa3bc2aaa90>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [20]:
rnd_search_cv_valid_on_train.best_params_

{'learning_rate': 0.025282146714733115, 'n_hidden': 3, 'n_neurons': 69}

In [21]:
rnd_search_cv_valid_on_train.best_score_

-0.29801326291659697

In [22]:
rnd_search_cv_valid_on_train.best_estimator_.model.evaluate(X_valid, y_valid)

3870/3870 [==============================] - 0s 31us/sample - loss: 0.2667


0.2666868191279798

In [23]:
rnd_search_cv.best_estimator_.model.evaluate(X_valid, y_valid)

3870/3870 [==============================] - 0s 31us/sample - loss: 0.2840


0.28398678107933173